In [ ]:
# # ! pip install --user numpy 
# from keras.models import load_model
# import numpy as np
# from numpy import load
# from numpy import expand_dims
# from numpy import asarray
# from numpy import savez_compressed
# from os import listdir
# from os.path import isdir
# from PIL import Image
# from mtcnn.mtcnn import MTCNN
# import cv2
# from numpy import load
# from sklearn.metrics import accuracy_score
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import Normalizer
# from sklearn.svm import SVC
# from random import choice
# from numpy import load
# from numpy import expand_dims
# from matplotlib import pyplot

# model = load_model('facenet_keras.h5')
# # summarize input and output shape
# print(model.inputs)
# print(model.outputs)

In [ ]:
# # to extract a single face from a photograph
# def extract_face(image_input, req_size=(160,160)):
#     # load image from file
#     image = Image.open(image_input)
#     # convert to RGB, if needed
#     image = image.convert('RGB')
#     # convert to array
#     pixels = asarray(image)
#     # create the detector, using default weights
#     detector = MTCNN()
#     # detect faces in the image
#     results = detector.detect_faces(pixels)
#     # extract the bounding box from the first face
#     x1, y1, width, height = results[0]['box']
#     # bug fix
#     x1, y1 = abs(x1), abs(y1)
#     x2, y2 = x1 + width, y1 + height
#     # extract the face
#     face = pixels[y1:y2, x1:x2]
#     # resize pixels to the model size
#     image = Image.fromarray(face)
#     image = image.resize(req_size)
#     face_array = asarray(image)
#     return face_array

In [1]:
# ! pip install --user numpy 
from keras.models import load_model
import numpy as np
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from os import listdir
from os.path import isdir
from PIL import Image
from mtcnn.mtcnn import MTCNN
import cv2
from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from random import choice
from numpy import load
from numpy import expand_dims
from matplotlib import pyplot

model = load_model('facenet_keras.h5')
# summarize input and output shape
print(model.inputs)
print(model.outputs)

Using TensorFlow backend.


[<tf.Tensor 'input_1:0' shape=(None, 160, 160, 3) dtype=float32>]
[<tf.Tensor 'Bottleneck_BatchNorm/cond/Identity:0' shape=(None, 128) dtype=float32>]


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
# to extract a single face from a photograph
def extract_face(image_input, req_size=(160,160)):
    # load image from file
    image = Image.open(image_input)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(req_size)
    face_array = asarray(image)
    return face_array

In [4]:
trainX, trainy = list(), list()
# enumerate folders, on per class
for subdir in listdir('C:/Users/Asus/Downloads/train'):
    faces = list()
    # path
    path = 'C:/Users/Asus/Downloads/train'+ '/' + subdir + '/'
    if not isdir(path):
        continue
    for filename in listdir(path):
        # path
        file_path = path + filename
        try:
            face = extract_face(file_path)
        except IndexError:
            pass
#         print(file_path)
        faces.append(face)
    labels = [subdir for _ in range(len(faces))]
    print('>loaded %d examples for class: %s' % (len(faces), subdir))
    # store
    trainX.extend(faces)
    trainy.extend(labels)
#         faces.append(face)

>loaded 14 examples for class: other
>loaded 15 examples for class: slim_shady


In [5]:
testX, testy = list(), list()
# enumerate folders, on per class
for subdir in listdir('C:/Users/Asus/Downloads/val'):
    faces = list()
    # path
    path = 'C:/Users/Asus/Downloads/val'+ '/' + subdir + '/'
    if not isdir(path):
        continue
    for filename in listdir(path):
        # path
        file_path = path + filename
        try:
            face = extract_face(file_path)
        except IndexError:
            pass
#         print(file_path)
        faces.append(face)
    labels = [subdir for _ in range(len(faces))]
    print('>loaded %d examples for class: %s' % (len(faces), subdir))
    # store
    testX.extend(faces)
    testy.extend(labels)
#         faces.append(face)

>loaded 5 examples for class: other
>loaded 7 examples for class: slim_shady


In [6]:
# save arrays to one file in compressed format
savez_compressed('faces-dataset.npz', trainX, trainy, testX, testy)

In [7]:
data = load('faces-dataset.npz',allow_pickle=True)
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

Loaded:  (29, 160, 160, 3) (29,) (12, 160, 160, 3) (12,)


In [8]:
# load the facenet model
model = load_model('facenet_keras.h5')
print('Loaded Model')

Loaded Model


In [9]:
# get the face embedding for one face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]

In [10]:
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)

(29, 128)


In [11]:
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
    embedding = get_embedding(model, face_pixels)
    newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)
# save arrays to one file in compressed format
savez_compressed('faces-embeddings.npz', newTrainX, trainy, newTestX, testy)

(12, 128)


In [12]:
data = load('faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))

Dataset: train=29, test=12


In [13]:
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)

In [14]:
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)

In [15]:
model = SVC(kernel='linear')
model.fit(trainX, trainy)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [16]:
import pickle
filename = 'linearSVC.sav'
pickle.dump(model, open(filename, 'wb'))

In [17]:
model = pickle.load(open(filename, 'rb'))
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Accuracy: train=100.000, test=100.000


In [18]:
data = load('faces-dataset.npz')
testX_faces = data['arr_2']

In [19]:
selection = choice([i for i in range(testX.shape[0])])
random_face_pixels = testX_faces[selection]
random_face_emb = testX[selection]
random_face_class = testy[selection]
random_face_name = out_encoder.inverse_transform([random_face_class])

In [20]:
samples = expand_dims(random_face_emb, axis=0)
yhat_class = model.predict(samples)
# yhat_prob = model.predict_proba(samples)

In [21]:
class_index = yhat_class[0]
# class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_class)

In [22]:
print('Predicted: %s' % (predict_names[0]))  #class_probability
print('Expected: %s' % random_face_name[0])

Predicted: slim_shady
Expected: slim_shady


In [23]:
cv2.imshow('face',random_face_pixels)
cv2.waitKey(5000)
cv2.destroyAllWindows()

In [1]:
# ! pip install mtcnn
# ! pip install --upgrade --force-reinstall pillow

import cv2
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

# Helper libraries
import numpy as np
from numpy import asarray
from PIL import Image

from mtcnn.mtcnn import MTCNN
# print version
# print(mtcnn.__version__)
from keras_vggface.vggface import VGGFace

2.0.0


Using TensorFlow backend.


In [2]:
# to extract a single face from a photograph
def extract_face(image_input, req_size=(160,160)):
    img = cv2.imread(image_input)
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(img)
    # extract bounding box values
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = img[y1:y2, x1:x2]
    # resizing image
    image = Image.fromarray(face)
    image = image.resize(req_size)
    face_array = asarray(image)
    return face

In [3]:
# face = extract_face(r'C:\Users\Asus\Downloads\opencv-face-recognition\dataset\shady\00000000.jpg')
# cv2.imshow('face',face)
# cv2.waitKey(5000)
# cv2.destroyAllWindows()

In [4]:
# cap.release()
cv2.destroyAllWindows()

In [27]:
# in_encoder = Normalizer(norm='l2')
# model = pickle.load(open(filename, 'rb'))

cap = cv2.VideoCapture(0)
blank_image = np.zeros((224,224,3), np.uint8)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

while(True):
    ret, frame = cap.read()
    detector = MTCNN()
    results = detector.detect_faces(frame)
    if len(results) > 0:
        for result in results:
            try:
                x1, y1, width, height = result['box']
                x2, y2 = x1 + width, y1 + height
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),3)
                face = frame[y1:y2,x1:x2]
                image = np.resize(image,(160,160,3))  
#                 cv2.imshow('frame',image)
#                 embedding = get_embedding(model, face_pixels)
#                 image = in_encoder.transform(embedding)
#                 yhat_class = model.predict(image)
#                 class_index = yhat_class[0]
#                 predict_names = out_encoder.inverse_transform(yhat_class)
                print(predict_names)
#                 image = Image.fromarray(face)
#                 image = image.resize((160,160))
#                 face_array = asarray(image)
#                 embedding = in_encoder.transform(embedding)
#                 embedding = get_embedding(model, face_array)
#                 samples = expand_dims(embedding, axis=0)
#                 yhat_class = model.predict(samples)
#                 predict_names = out_encoder.inverse_transform(yhat_class)
#                 print(predict_names)
#                 if predict_names[0] == 'slim_shady':
#                     cv2.putText(image, 'slim_shady', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
#                 else:
#                     cv2.imshow('frame',frame)
#                 out.write(frame)
                cv2.imshow('frame',frame)
            except:
#                 cv2.imshow('face',blank_image)
                frame = cv2.putText(frame, 'No Person', (10,10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
                
    else:
#         frame = cv2.putText(frame, 'No Person', (10,10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
# out.release()
cv2.destroyAllWindows()

In [ ]:
# face = extract_face('C:/Users/Asus/Downloads/data/Images\shady_00000011.jpg')
# cv2.imshow('face',face)
# cv2.waitKey(5000)
# cv2.destroyAllWindows()

In [ ]:
# for img_name in os.listdir('Test_Images/'):
#     img=cv2.imread(path+'/Test_Images/'+img_name)
#     extracted_face = extract_face(img)
#     cv2.imwrite(path+'/Test_Images/crop_img.jpg',extracted_face)
    
#     # Get Embeddings
#     crop_img=load_img(path+'/Test_Images/crop_img.jpg',target_size=(224,224))
#     crop_img=img_to_array(crop_img)
#     crop_img=np.expand_dims(crop_img,axis=0)
#     crop_img=preprocess_input(crop_img)
#     img_encode=vgg_face(crop_img)

#     # Make Predictions
#     embed=K.eval(img_encode)
#     person=classifier_model.predict(embed)
#     name=person_rep[np.argmax(person)]
#     os.remove(path+'/Test_Images/crop_img.jpg')
#     cv2.rectangle(img,(left,top),(right,bottom),(0,255,0), 2)
#     img=cv2.putText(img,name,(left,top-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,255),2,cv2.LINE_AA)
#     img=cv2.putText(img,str(np.max(person)),(right,bottom+10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
#   # Save images with bounding box,name and accuracy 
#     cv2.imwrite(path+'/Predictions/'+img_name,img)

In [ ]:
# os.mkdir(path+'/Predictions')

In [ ]:
# person_rep={0:'owen',1:'shady'}

In [ ]:
# def plot(img):
#     plt.figure(figsize=(8,4))
#     plt.imshow(img[:,:,::-1])
#     plt.show()

In [ ]:
# test_images_path=path+'/Test_Images/'

In [ ]:
# # Load saved model
# classifier_model=tf.keras.models.load_model('C:/Users/Asus/Downloads/face_classifier_model.h5')



In [ ]:
# # Save model for later use
# tf.keras.models.save_model(classifier_model,'C:/Users/Asus/Downloads/face_classifier_model.h5')

In [ ]:
# classifier_model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test))

In [ ]:
# # Softmax regressor to classify images based on encoding 
# classifier_model=Sequential()
# classifier_model.add(Dense(units=100,input_dim=x_train.shape[1],kernel_initializer='glorot_uniform'))
# classifier_model.add(BatchNormalization())
# classifier_model.add(Activation('tanh'))
# classifier_model.add(Dropout(0.3))
# classifier_model.add(Dense(units=10,kernel_initializer='glorot_uniform'))
# classifier_model.add(BatchNormalization())
# classifier_model.add(Activation('tanh'))
# classifier_model.add(Dropout(0.2))
# classifier_model.add(Dense(units=6,kernel_initializer='he_uniform'))
# classifier_model.add(Activation('softmax'))
# classifier_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='nadam',metrics=['accuracy'])

In [ ]:
# # Load saved data
# x_train=np.load('train_data.npy')
# y_train=np.load('train_labels.npy')
# x_test=np.load('test_data.npy')
# y_test=np.load('test_labels.npy')

In [ ]:
# # Save test and train data for later use
# np.save('train_data',x_train)
# np.save('train_labels',y_train)
# np.save('test_data',x_test)
# np.save('test_labels',y_test)

In [ ]:
# x_test=np.array(x_test)
# y_test=np.array(y_test)

In [ ]:
# #Prepare Test Data
# x_test=[]
# y_test=[]
# person_folders=os.listdir(path+'/Test_Images_crop/')
# for i,person in enumerate(person_folders):
#     image_names=os.listdir('Test_Images_crop/'+person+'/')
#     for image_name in image_names:
#         img=load_img(path+'/Test_Images_crop/'+person+'/'+image_name,target_size=(224,224))
#         img=img_to_array(img)
#         img=np.expand_dims(img,axis=0)
#         img=preprocess_input(img)
#         img_encode=vgg_face(img)
#         x_test.append(np.squeeze(K.eval(img_encode)).tolist())
#         y_test.append(i)

In [ ]:
# x_train=np.array(x_train)
# y_train=np.array(y_train)

In [ ]:
# person_rep

In [ ]:
# #Prepare Training Data
# x_train=[]
# y_train=[]
# person_folders=os.listdir(path+'/Images_crop/')
# person_rep=dict()
# for i,person in enumerate(person_folders):
#     person_rep[i]=person
#     image_names=os.listdir('Images_crop/'+person+'/')
#     for image_name in image_names:
#         img=load_img(path+'/Images_crop/'+person+'/'+image_name,target_size=(224,224))
#         img=img_to_array(img)
#         img=np.expand_dims(img,axis=0)
#         img=preprocess_input(img)
#         img_encode=vgg_face(img)
#         x_train.append(np.squeeze(K.eval(img_encode)).tolist())
#         y_train.append(i)

In [ ]:
# # Remove Last Softmax layer and get model upto last flatten layer with outputs 2622 units
# vgg_face=Model(inputs=model.layers[0].input,outputs=model.layers[-2].output)

In [ ]:
# model.summary()

In [ ]:
# # Load VGG Face model weights
# model.load_weights('vgg_face_weights.h5')

In [ ]:
# #Define VGG_FACE_MODEL architecture
# model = Sequential()
# model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
# model.add(Convolution2D(64, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(128, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(256, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(256, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))
# model.add(Convolution2D(4096, (7, 7), activation='relu'))
# model.add(Dropout(0.5))
# model.add(Convolution2D(4096, (1, 1), activation='relu'))
# model.add(Dropout(0.5))
# model.add(Convolution2D(2622, (1, 1)))
# model.add(Flatten())
# model.add(Activation('softmax'))

In [ ]:
# for file_name in image_path_names:
#     extracted_face = extract_face(file_name)
#     img_path=path+'/Test_Images_crop/'+file_name.split('/')[-1].split('_')[0]+'/'+file_name.split('/')[-1]
#     cv2.imwrite(img_path,extracted_face)

In [ ]:
# os.mkdir(path+'/Test_Images_crop/')

In [ ]:
# # Get Image names for testing
# test_image_path_names=[]
# for file_name in glob.glob(path+'/Images_test/*.jpg'):
#     test_image_path_names.append(file_name)

In [20]:
import numpy as np
import cv2
from PIL import Image
import os

In [21]:
dimen = 128

dir_path = r'C:\Users\Asus\Downloads\data\images'
out_path = r'C:\Users\Asus\Downloads\data\processed_data'

In [22]:
sub_dir_list = os.listdir(dir_path)
images = list()
labels = list()
# print(sub_dir_list)

In [23]:
for i in range(len(sub_dir_list)):
    label = i
    image_names = os.listdir(os.path.join(dir_path ,sub_dir_list[i]))
    for image_path in image_names:
        path = os.path.join(dir_path ,sub_dir_list[i],image_path)
        try :
            image = Image.open(path)
            resize_image = image.resize((dimen, dimen))
            array_ = list()
            for x in range(dimen):
                sub_array = list()
                for y in range(dimen):
                    sub_array.append(resize_image.load()[x, y])
                array_.append(sub_array)
            image_data = np.array(array_)
            image = np.array(np.reshape(image_data, (dimen, dimen, 3))) / 255
            images.append(image)
            labels.append(label)
        except:
            print( 'WARNING : File {} could not be processed.'.format( path ) )

In [24]:
labels

[0, 0, 0, 1, 1, 1]

In [25]:
images = np.array( images )

images[0].shape

(128, 128, 3)

In [26]:
samples_1 = list()
samples_2 = list()
labels = list()
for i in range(6) :
    for j in range(6) :
        samples_1.append(images[i])
        samples_2.append(images[j])
        if i < 3 :
            if j < 3 :
                labels.append(1)
            else:
                labels.append(0)
        else :
            if j > 2 :
                labels.append(1)
            else:
                labels.append(0)

X1 = np.array( samples_1  )
X2 = np.array( samples_2 )
Y = np.array( labels )

In [36]:
X1.shape

(36, 128, 128, 3)

In [32]:
np.save( '{}/x1.npy'.format( out_path ), X1 )
np.save( '{}/x2.npy'.format( out_path ), X2 )
np.save( '{}/y.npy'.format( out_path ) , Y )

In [81]:
from tensorflow.python.keras import models , optimizers , losses ,activations , callbacks
from tensorflow.python.keras.layers import *
import tensorflow.python.keras.backend as K
from PIL import Image
import tensorflow.keras
import time
import os
import numpy as np

In [54]:
# class Recognizer(object):
#     def __int__(self):
#         tf.logging.set_verbosity(tf.logging.ERROR)
#         self.__DIMEN = 128

#         input_shape = (self.__DIMEN ** 2) * 3
#         convolution_shape = ( self.__DIMEN , self.__DIMEN , 3 )
#         kernel_size_1 = ( 4 , 4 )
#         kernel_size_2 = ( 3 , 3 )
#         pool_size_1 = ( 3 , 3 )
#         pool_size_2 = ( 2 , 2 )
#         strides = 1

#         seq_conv_model = [

#             Reshape( input_shape=input_shape , target_shape=convolution_shape),

#             Conv2D( 32, kernel_size=kernel_size_1 , strides=strides , activation=activations.leaky_relu ),
#             Conv2D( 32, kernel_size=kernel_size_1, strides=strides, activation=activations.leaky_relu),
#             MaxPooling2D(pool_size=pool_size_1, strides=strides ),

#             Conv2D( 64, kernel_size=kernel_size_2 , strides=strides , activation=activations.leaky_relu ),
#             Conv2D( 64, kernel_size=kernel_size_2 , strides=strides , activation=activations.leaky_relu ),
#             MaxPooling2D(pool_size=pool_size_2 , strides=strides),

#             Flatten(),

#             Dense( 64 , activation=activations.sigmoid )

#         ]

#         seq_model = tf.keras.Sequential( seq_conv_model )

#         input_x1 = Input( shape=input_shape )
#         input_x2 = Input( shape=input_shape )

#         output_x1 = seq_model( input_x1 )
#         output_x2 = seq_model( input_x2 )

#         distance_euclid = Lambda( lambda tensors : K.abs( tensors[0] - tensors[1] ))( [output_x1 , output_x2] )
#         outputs = Dense( 1 , activation=activations.sigmoid) ( distance_euclid )
#         self.__model = models.Model( [ input_x1 , input_x2 ] , outputs )

#         self.__model.compile( loss=losses.binary_crossentropy , optimizer=optimizers.Adam(lr=0.0001))

#     def fit(self, X, Y ,hyperparameters):
#         initial_time = time.time()
#         self.__model.fit( X  , Y ,
#                          batch_size=hyperparameters[ 'batch_size' ] ,
#                          epochs=hyperparameters[ 'epochs' ] ,
#                          callbacks=hyperparameters[ 'callbacks'],
#                          validation_data=hyperparameters[ 'val_data' ]
#                          )
#         final_time = time.time()
#         eta = ( final_time - initial_time )
#         time_unit = 'seconds'
#         if eta >= 60 :
#             eta = eta / 60
#             time_unit = 'minutes'
#         self.__model.summary( )
#         print( 'Elapsed time acquired for {} epoch(s) -> {} {}'.format( hyperparameters[ 'epochs' ] , eta , time_unit ) )

#     def evaluate(self , test_X , test_Y  ) :
#         return self.__model.evaluate(test_X, test_Y)


#     def predict(self, X  ):
#         predictions = self.__model.predict( X  )
#         return predictions


#     def summary(self):
#         self.__model.summary()

#     def save_model(self , file_path ):
#         self.__model.save(file_path )


#     def load_model(self , file_path ):
#         self.__model = models.load_model(file_path)
        
# dimen = 128
# input_shape = (self.__DIMEN ** 2) * 3
# convolution_shape = ( self.__DIMEN , self.__DIMEN , 3 )
# kernel_size_1 = (4,4)
# kernel_size_2 = (3,3)
# pool_size_1 = (3,3)
# pool_size_2 = (2,2)
# strides = 1

# seq_conv_model = [

#     Reshape( input_shape=input_shape , target_shape=convolution_shape),

#     Conv2D(32, kernel_size=kernel_size_1 , strides=strides , activation=activations.leaky_relu),
#     Conv2D( 32, kernel_size=kernel_size_1, strides=strides, activation=activations.leaky_relu),
#     MaxPooling2D(pool_size=pool_size_1, strides=strides ),

#     Conv2D(64, kernel_size=kernel_size_2 , strides=strides , activation=activations.leaky_relu ),
#     Conv2D(64, kernel_size=kernel_size_2 , strides=strides , activation=activations.leaky_relu ),
#     MaxPooling2D(pool_size=pool_size_2 , strides=strides),

#     Flatten(),

#     Dense( 64 , activation=activations.sigmoid )

# ]

# seq_model = tf.keras.Sequential( seq_conv_model )

# input_x1 = Input( shape=input_shape )
# input_x2 = Input( shape=input_shape )

# output_x1 = seq_model( input_x1 )
# output_x2 = seq_model( input_x2 )

# distance_euclid = Lambda( lambda tensors : K.abs( tensors[0] - tensors[1] ))( [output_x1 , output_x2] )
# outputs = Dense( 1 , activation=activations.sigmoid) ( distance_euclid )
# self.__model = models.Model( [ input_x1 , input_x2 ] , outputs )

# self.__model.compile( loss=losses.binary_crossentropy , optimizer=optimizers.Adam(lr=0.0001))

In [46]:
data_dimension = 128

X1 = np.load(r'C:\Users\Asus\Downloads\data\processed_data\x1.npy')
X2 = np.load(r'C:\Users\Asus\Downloads\data\processed_data\x2.npy')
Y = np.load(r'C:\Users\Asus\Downloads\data\processed_data\y.npy')

X1 = X1.reshape( ( X1.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )
X2 = X2.reshape( ( X2.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )

print( X1.shape )
print( X2.shape )
print( Y.shape )


(36, 49152)
(36, 49152)
(36,)


In [47]:
# recognizer = Recognizer()

In [48]:
parameters = {
    'batch_size' : 6 ,
    'epochs' : 5 ,
    'callbacks' : None , # [ TensorBoard( log_dir='logs/{}'.format( time.time() ) ) ] ,
    'val_data' : None
}

In [53]:
# recognizer.fit( [ X1 , X2 ], Y, hyperparameters=parameters)
# recognizer.save_model('models/model.h5')

In [84]:
# Keras Sequential Model
tf.random.set_seed(100)

tf.keras.backend.clear_session()
leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.3)


dimen = 128
input_shape = ((dimen ** 2) * 3,)
convolution_shape = (128 , 128 , 3 )
kernel_size_1 = (4,4)
kernel_size_2 = (3,3)
pool_size_1 = (3,3)
pool_size_2 = (2,2)
strides = 1

model = None
seq_model = keras.Sequential([
    keras.layers.Reshape(input_shape=input_shape, target_shape = (128 , 128 , 3 )),
    keras.layers.Conv2D( 32, kernel_size=kernel_size_1 , strides=strides , activation=leaky_relu ),
    keras.layers.Conv2D( 32, kernel_size=kernel_size_1, strides=strides, activation=leaky_relu),
    keras.layers.MaxPooling2D(pool_size=pool_size_1, strides=strides),
    keras.layers.Conv2D( 64, kernel_size=kernel_size_2 , strides=strides , activation=leaky_relu ),
    keras.layers.Conv2D( 64, kernel_size=kernel_size_2 , strides=strides , activation=leaky_relu ),
    keras.layers.MaxPooling2D(pool_size=pool_size_2 , strides=strides),
    keras.layers.Flatten(),
    keras.layers.Dense(64)
    ])
    

# seq_model = tf.keras.Sequential(seq_conv_model)

input_x1 = Input(shape=input_shape)
input_x2 = Input(shape=input_shape)

output_x1 = seq_model(input_x1)
output_x2 = seq_model(input_x2 )

distance_euclid = Lambda( lambda tensors : K.abs( tensors[0] - tensors[1] ))( [output_x1 , output_x2] )
outputs = Dense( 1 , activation=activations.sigmoid) ( distance_euclid )
model = models.Model( [ input_x1 , input_x2 ] , outputs )

model.compile( loss=losses.binary_crossentropy , optimizer=optimizers.Adam(lr=0.0001))

initial_time = time.time()
model.fit(X ,Y ,
            batch_size=hyperparameters[ 'batch_size' ] ,
            epochs=hyperparameters[ 'epochs' ] ,
            callbacks=hyperparameters[ 'callbacks'],
            validation_data=hyperparameters[ 'val_data' ])

final_time = time.time()
eta = ( final_time - initial_time )
time_unit = 'seconds'
if eta >= 60 :
    eta = eta / 60
    time_unit = 'minutes'
model.summary()
print('Elapsed time acquired for {} epoch(s) -> {} {}'.format( hyperparameters[ 'epochs' ] , eta , time_unit))
model.save(r'C:\Users\Asus\Downloads\data\model.h5')

AttributeError: 'tuple' object has no attribute 'layer'

In [ ]:
def prepare_images_from_dir(dir_path , flatten=True):
    images = list()
    images_names = os.listdir( dir_path )
    for imageName in images_names :
        image = Image.open(dir_path + imageName)
        resize_image = image.resize((dimen, dimen))
        array = list()
        for x in range(dimen):
            sub_array = list()
            for y in range(dimen):
                sub_array.append(resize_image.load()[x, y])
            array.append(sub_array)
        image_data = np.array(array)
        image = np.array(np.reshape(image_data,(dimen, dimen, 3)))
        images.append(image)

    if flatten :
        images = np.array(images)
        return images.reshape( ( images.shape[0], dimen**2 * 3  ) ).astype( np.float32 )
    else:
        return np.array(images)

In [ ]:
custom_images = recognizer.prepare_images_from_dir( 'custom_images/'  )
class_1_images = recognizer.prepare_images_from_dir( 'images/p1/' )
class_2_images = recognizer.prepare_images_from_dir( 'images/p2/' )

In [ ]:
scores = list()
labels = list()
for image in custom_images:
    label = list()
    score = list()
    for sample in class_1_images :
        image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
        score.append( recognizer.predict( [ image , sample ])[0] )
        label.append( 0 )
    for sample in class_2_images :
        image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
        score.append( recognizer.predict( [ image , sample ])[0] )
        label.append( 1 )
    labels.append( label )
    scores.append( score )

scores = np.array( scores )
labels = np.array( labels )

for i in range( custom_images.shape[0] ) :
    index = np.argmax( scores[i] )
    label_ = labels[i][index]
    print( 'IMAGE {} is {} with confidence of {}'.format( i+1  , label_ , scores[i][index][0])

In [56]:
(dimen ** 2) * 3

49152

In [60]:
import keras

Using TensorFlow backend.


In [85]:
from keras.layers import Input

In [3]:
def prepare_images_from_dir(dir_path , flatten=True) :
    images = list()
    folder_list = ['Shady', 'Unknown', 'sample']
    images_names = os.listdir( dir_path )
    for imageName in images_names :
        if imageName in images_names:
            image = Image.open(dir_path + imageName)
            resize_image = image.resize((128, 128))
            array = list()
            for x in range(128):
                sub_array = list()
            for y in range(128):
                sub_array.append(resize_image.load()[x, y])
            array.append(sub_array)
            image_data = np.array(array)
            image = np.array(np.reshape(image_data,(128, 128, 3)))
            images.append(image)
        
    if flatten :
        images = np.array(images)
        return images.reshape( ( images.shape[0]  , (128**2) * 3  ) ).astype( np.float32 )
    else:
        return np.array(images)